# 使用者相似度去推薦食譜

In [16]:
import numpy as np
import math

# 使用者資料 （隨便生成的）
**資料格式** : (對第一份食譜的評價, 對第二份食譜的評價, 對第三份食譜的評價, 對第四份食譜的評價, 對第五份食譜的評價) 

說明: 滿星五顆，最低一顆 nan代表使用者沒有評分 主食材暫定權重為2~2.5 副食材暫定權重為1~1.5 佐料暫定權重為 < 0.5


In [17]:
user_love_list = [
    (3,2,1,2,3),
    (1,1,1,1,1), # 極端資料
    (3,2,3,math.nan,2),
    (3,2,3,2,1),
    (1,math.nan,1,1,5),
    (1,2,1,1,1),
    (3,2,2,2,1),
    (1,1,1,1,2)
]

# comment this
user_love_list +=[
    (3,2,3,3,2), #
    (3,2,3,5,2),
    (3,2,3,4,2),
    (3,2,3,3,2),
    (3,2,3,3,2)
]



recipe_ingr=[
    (0.0,2.1,0.0,2.0,1.0,0.5), ##
    (1.0,2.2,0.0,0.0,1.0,0.3),
    (0.0,2.1,2.2,2.0,1.0,0.1), ##
    (0.0,2.3,0.0,0.0,1.0,0.5), ##
    (0.0,0.0,0.0,2.0,1.0,0.5)
]

In [18]:
# 標準化所有資料

std_user_love = []

for arr in user_love_list:
    arr = (arr - np.nanmean(arr)) / np.nanstd(arr)
    std_user_love.append(arr)

std_user_love_np = np.array(std_user_love)
std_user_love_np

/var/folders/b8/4_1zgdyd5p98vtr649zl136c0000gn/T/ipykernel_17569/3331034335.py:6: RuntimeWarning: invalid value encountered in divide
  arr = (arr - np.nanmean(arr)) / np.nanstd(arr)


array([[ 1.06904497, -0.26726124, -1.60356745, -0.26726124,  1.06904497],
       [        nan,         nan,         nan,         nan,         nan],
       [ 1.        , -1.        ,  1.        ,         nan, -1.        ],
       [ 1.06904497, -0.26726124,  1.06904497, -0.26726124, -1.60356745],
       [-0.57735027,         nan, -0.57735027, -0.57735027,  1.73205081],
       [-0.5       ,  2.        , -0.5       , -0.5       , -0.5       ],
       [ 1.58113883,  0.        ,  0.        ,  0.        , -1.58113883],
       [-0.5       , -0.5       , -0.5       , -0.5       ,  2.        ],
       [ 0.81649658, -1.22474487,  0.81649658,  0.81649658, -1.22474487],
       [ 0.        , -0.91287093,  0.        ,  1.82574186, -0.91287093],
       [ 0.26726124, -1.06904497,  0.26726124,  1.60356745, -1.06904497],
       [ 0.81649658, -1.22474487,  0.81649658,  0.81649658, -1.22474487],
       [ 0.81649658, -1.22474487,  0.81649658,  0.81649658, -1.22474487]])

# 相似度計算
![](./img.png)

根據論文 可以使用上面的公式，另外再加上一個閥值來過濾相似度，另外也可以做top-k的過濾

In [19]:
threshold = 0.7

In [20]:
# 計算使用者相似度
similar_np = []
target = std_user_love_np[2]
for arr in std_user_love_np:
    child = 0
    mom1 = 0
    mom2 = 0
    for i in range(len(arr)):
        temp = (arr[i]*target[i])
        if not math.isnan(temp):
            child += temp
        else:
            continue
        mom1 += arr[i]*arr[i]
        mom2 += target[i]*target[i]

    similar_np.append(child / (np.sqrt(mom1) * np.sqrt(mom2)))

similar_np

/var/folders/b8/4_1zgdyd5p98vtr649zl136c0000gn/T/ipykernel_17569/3996714476.py:17: RuntimeWarning: invalid value encountered in double_scalars
  similar_np.append(child / (np.sqrt(mom1) * np.sqrt(mom2)))


[-0.30096463271442303,
 nan,
 1.0,
 0.9028938981432689,
 -0.8703882797784893,
 -0.5735393346764043,
 0.7071067811865476,
 -0.5735393346764044,
 0.9805806756909201,
 0.7071067811865476,
 0.8574929257125442,
 0.9805806756909201,
 0.9805806756909201]

# 把相似度高的相關使用者挑出來 再根據權重做標準化 得出預測的結果

In [21]:
predict_np = target
predict_np[np.isnan(predict_np)] = np.nanmean(predict_np) # 取代nan變成平均值 讓nan不會影響這個vector
count = 0;
for i in range(len(similar_np)):
    if i == 2: # 自己不用計算
        continue
    if similar_np[i] > threshold: 
        predict_np += std_user_love_np[i] * similar_np[i]
        count+=similar_np[i]
        
target

array([ 5.7143651 , -6.40638773,  4.59633111,  4.82665596, -8.73096444])

# 對食譜的評價預測
(再次標準化)

In [29]:
# predict_np = (predict_np - predict_np.mean()) / predict_np.std()
predict_np

array([ 0.91656343, -1.02756135,  0.73723483,  0.7741781 , -1.40041502])

# 我們可以從上面的數據得知使用者可能對這些食譜的喜歡以及厭惡程度
把他帶入食材的部分


In [23]:
recipe_ingr_np = np.array(recipe_ingr)

recipe_ingr_np

array([[0. , 2.1, 0. , 2. , 1. , 0.5],
       [1. , 2.2, 0. , 0. , 1. , 0.3],
       [0. , 2.1, 2.2, 2. , 1. , 0.1],
       [0. , 2.3, 0. , 0. , 1. , 0.5],
       [0. , 0. , 0. , 2. , 1. , 0.5]])

In [24]:
i_love = np.zeros(len(recipe_ingr[0]),dtype=np.float64)
for i in range(len(predict_np)):
    i_love += recipe_ingr_np[i] * predict_np[i]

i_love

array([-1.02756135,  2.99295102,  1.62191663,  0.50676649,  0.        ,
       -0.08938166])

In [25]:
np.matmul(i_love,recipe_ingr_np.T)

array([ 7.25403929,  5.53011641, 10.85800855,  6.83909652,  0.96884215])

---

# 沒標準化

## 沒有標準化的用戶評分是沒辦法看出這個用戶對食譜的喜愛程度的
可以從下方的數據看出來 很明顯數字出來是不能用的

In [26]:
user_love_np = np.array(user_love_list)

similar_np = []
target = user_love_np[2]
for arr in user_love_np:
    child = 0
    mom1 = 0
    mom2 = 0
    for i in range(len(arr)):
        temp = (arr[i]*target[i])
        if not math.isnan(temp):   
            child += temp
        else:
            continue
        mom1 += arr[i]*arr[i]
        mom2 += target[i]*target[i]

    similar_np.append(child / (np.sqrt(mom1) * np.sqrt(mom2)))

similar_np

[0.8996469021204839,
 0.9805806756909202,
 1.0000000000000002,
 0.981432984131437,
 0.6564879518897745,
 0.8894991799933214,
 0.9707253433941511,
 0.8894991799933214,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002]

# 可以發現第一筆與第二筆的資料的相似度很高 
# 這是因為每個人的評分標準不一樣 
# 如果沒有標準化只會讓數據失準
(下方為第一筆與第三筆資料的差別)


In [27]:
user_love_list[1]

(1, 1, 1, 1, 1)

In [28]:
user_love_list[2]

(3, 2, 3, nan, 2)